In [ ]:
import pandas as pd

In [ ]:
# prompt: read csv from here: https://github.com/jinshijian/SoilHealthDB/blob/master/data/SoilHealthDB_V2.csv

# Download the file first
!wget https://raw.githubusercontent.com/jinshijian/SoilHealthDB/master/data/SoilHealthDB_V2.csv

# Read the downloaded CSV file
df = pd.read_csv('SoilHealthDB_V2.csv', encoding='latin-1')

# Display the first few rows to verify
print(df.head())

--2025-06-12 11:41:37--  https://raw.githubusercontent.com/jinshijian/SoilHealthDB/master/data/SoilHealthDB_V2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3545298 (3.4M) [application/octet-stream]
Saving to: ‘SoilHealthDB_V2.csv’

SoilHealthDB_V2.csv 100%[===================>]   3.38M  --.-KB/s    in 0.09s   

2025-06-12 11:41:37 (37.5 MB/s) - ‘SoilHealthDB_V2.csv’ saved [3545298/3545298]



<ipython-input-3-4129539283>:7: DtypeWarning: Columns (16,19,20,24,25,52,77,122,150,152,160,192,197,212,232,237,242,247,252,262,265,266) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SoilHealthDB_V2.csv', encoding='latin-1')


   StudyID ExperimentID Author_F Author_G  YearPublication  SamplingYear  \
0        1        1-Jan    Abawi     G.S.             2000           NaN   
1        1        1-Jan    Abawi     G.S.             2000           NaN   
2        1        2-Jan    Abawi     G.S.             2000           NaN   
3        1        1-Jan    Abawi     G.S.             2000           NaN   
4        1        4-Jan    Abawi     G.S.             2000           NaN   

                Journal SiteInfor Country  Latitude  ...  MBN_C_SD  MBN_T_SD  \
0  Applied Soil Ecology  New York     USA     40.71  ...       NaN       NaN   
1  Applied Soil Ecology  New York     USA     40.71  ...       NaN       NaN   
2  Applied Soil Ecology  New York     USA     40.71  ...       NaN       NaN   
3  Applied Soil Ecology  New York     USA     40.71  ...       NaN       NaN   
4  Applied Soil Ecology  New York     USA     40.71  ...       NaN       NaN   

   38.MBN_Comments  Microelement  SQI  ESS Texture_C          

## cleaning

Remove all columns containing 'comment'

In [ ]:
# prompt: remove all columns containing 'comment'

columns_to_drop = [col for col in df.columns if 'comment' in col.lower()]
df_cleaned = df.drop(columns=columns_to_drop)

# Display the first few rows of the cleaned DataFrame
print(df_cleaned.head())

   StudyID ExperimentID Author_F Author_G  YearPublication  SamplingYear  \
0        1        1-Jan    Abawi     G.S.             2000           NaN   
1        1        1-Jan    Abawi     G.S.             2000           NaN   
2        1        2-Jan    Abawi     G.S.             2000           NaN   
3        1        1-Jan    Abawi     G.S.             2000           NaN   
4        1        4-Jan    Abawi     G.S.             2000           NaN   

                Journal SiteInfor Country  Latitude  ...  MBC_C_SD  MBC_T_SD  \
0  Applied Soil Ecology  New York     USA     40.71  ...       NaN       NaN   
1  Applied Soil Ecology  New York     USA     40.71  ...       NaN       NaN   
2  Applied Soil Ecology  New York     USA     40.71  ...       NaN       NaN   
3  Applied Soil Ecology  New York     USA     40.71  ...       NaN       NaN   
4  Applied Soil Ecology  New York     USA     40.71  ...       NaN       NaN   

   MBN_C  MBN_T  MBN_C_SD  MBN_T_SD Microelement  SQI  ESS Tex

Remove records that don't have organic carbon data

In [ ]:
# prompt: remove all records that do not contain values for OC__T or OC_C

df_filtered = df.dropna(subset=['OC_T', 'OC_C'])

print(df_filtered.head())
print(f"Original number of rows: {len(df)}")
print(f"Number of rows after filtering: {len(df_filtered)}")


    StudyID ExperimentID       Author_F  Author_G  YearPublication  \
11        2        1-Feb         Bandic      Anna             1999   
12        2        2-Feb         Bandic      Anna             1999   
55        4        1-Apr  Blanco-Canqui  Humberto             2010   
56        4        2-Apr  Blanco-Canqui  Humberto             2010   
57        4        1-Apr  Blanco-Canqui  Humberto             2010   

    SamplingYear                                     Journal  \
11        1994.0               Soil Biology and Biochemistry   
12        1994.0               Soil Biology and Biochemistry   
55        2002.0  Soil and Water Management and Conservation   
56        2002.0  Soil and Water Management and Conservation   
57        2002.0  Soil and Water Management and Conservation   

                                            SiteInfor Country  Latitude  ...  \
11  North  Willamette  Research  and Extension  Ce...     USA     45.23  ...   
12  North  Willamette  Research  a

Make new df with only fields that will be input by user about their property (as these are the only ones that can be used to predict the soil carbon of their property).  temperature, precipitation, cover crop (does it use cover crop), landuse type, type of tillage, crop, Elevation. Make sure all are numeric (i.e. use onehotencoder). Not study found texture is largest indicator, but most records don't have texture recorded. SoilFamily might be a substitute? Need to use ControlDescription - combine with cover crop field to indicate whether cover crop or not. i.e. fallow, lupin etc. in one field

Fields:
- Tannual : annual temperature
- Pannual : annual precipitation


Fields to be created or used with onehotencoder
- Fertilization _C _T : removed as incomplete and messy
- Tillage _C _T
- CoverCropGroup (LG is legumes, needs to be cleaned)
- GrainCropGroup
- Conservation_Type

In [ ]:
# prompt: create new df with fields Tannual, Pannual, Fertilization_C, Fertilization_T, Tillage_C, Tillage_T, CoverCropGroup, GrainCrop, Conservation_Type

# Select the desired columns to create the new DataFrame
df_new = df_filtered[['Country', 'Tannual', 'Pannual', 'OC_T', 'OC_C', 'Tillage_C', 'Tillage_T', 'CoverCropGroup', 'GrainCropGroup', 'Conservation_Type']].copy()

print(df_new.head())

   Country  Tannual  Pannual   OC_T   OC_C Fertilization_C Fertilization_T  \
11     USA      NaN      NaN  3.430  3.930          Varied          Varied   
12     USA      NaN      NaN  4.130  3.930          Varied          Varied   
55     USA      NaN      NaN  1.693  1.368   0 kg N per ha   0 kg N per ha   
56     USA      NaN      NaN  1.071  1.031   0 kg N per ha   0 kg N per ha   
57     USA      NaN      NaN  1.816  1.368   0 kg N per ha   0 kg N per ha   

     Tillage_C   Tillage_T CoverCropGroup GrainCropGroup Conservation_Type  
11         NaN         NaN            Rye            MTT                CC  
12         NaN         NaN         Legume            MTT                CC  
55  No tillage  No tillage         Legume             WO             CC-NT  
56  No tillage  No tillage         Legume             WO             CC-NT  
57  No tillage  No tillage         Legume             WO             CC-NT  


In [ ]:
# prompt: check how many rows df_new has

print(f"Number of rows in df_new: {len(df_new)}")

Number of rows in df_new: 1573


In [ ]:
# prompt: check how many na's df_new has for Tannual

print(f"Number of NA's in Tannual column: {df_new['Tannual'].isnull().sum()}")
print(f"Number of NA's in Pannual column: {df_new['Pannual'].isnull().sum()}")


Number of NA's in Tannual column: 1156
Number of NA's in Pannual column: 1095


In [ ]:
# prompt: remove records where Tannual is null

df_new_cleaned = df_new.dropna(subset=['Tannual'])

print(df_new_cleaned.head())
print(f"Number of rows after removing null Tannual: {len(df_new_cleaned)}")
print(f"Number of NA's in Tannual column after cleaning: {df_new_cleaned['Tannual'].isnull().sum()}")

    Country  Tannual  Pannual  OC_T  OC_C Fertilization_C Fertilization_T  \
424     USA     19.6    976.0  0.74  0.46   0 Kg N per ha   0 Kg N per ha   
425     USA     19.6    976.0  0.70  0.46   0 Kg N per ha   0 Kg N per ha   
426     USA     19.6    976.0  0.59  0.46   0 Kg N per ha   0 Kg N per ha   
427     USA     19.6    976.0  0.73  0.50  67 Kg N per ha  67 Kg N per ha   
428     USA     19.6    976.0  0.74  0.50  67 Kg N per ha  67 Kg N per ha   

                Tillage_C             Tillage_T CoverCropGroup GrainCropGroup  \
424  Conventional tillage  Conservation tillage             LL           Corn   
425  Conventional tillage  Conservation tillage         Legume           Corn   
426  Conventional tillage  Conservation tillage         Legume           Corn   
427  Conventional tillage  Conservation tillage             LL           Corn   
428  Conventional tillage  Conservation tillage         Legume           Corn   

    Conservation_Type  
424                CC  
42

In [ ]:
print(f"Number of NA's in Tannual column: {df_new_cleaned['Tannual'].isnull().sum()}")
print(f"Number of NA's in Pannual column: {df_new_cleaned['Pannual'].isnull().sum()}")
print(f"Number of NA's in Pannual column: {df_new_cleaned['CoverCropGroup'].isnull().sum()}")


Number of NA's in Tannual column: 0
Number of NA's in Pannual column: 0
Number of NA's in Pannual column: 0


In [ ]:
# prompt: remove the records with Pannual of null

df_new_cleaned = df_new_cleaned.dropna(subset=['Pannual'])

print(df_new_cleaned.head())
print(f"Number of rows after removing null Pannual: {len(df_new_cleaned)}")
print(f"Number of NA's in Pannual column after cleaning: {df_new_cleaned['Pannual'].isnull().sum()}")

    Country  Tannual  Pannual  OC_T  OC_C Fertilization_C Fertilization_T  \
424     USA     19.6    976.0  0.74  0.46   0 Kg N per ha   0 Kg N per ha   
425     USA     19.6    976.0  0.70  0.46   0 Kg N per ha   0 Kg N per ha   
426     USA     19.6    976.0  0.59  0.46   0 Kg N per ha   0 Kg N per ha   
427     USA     19.6    976.0  0.73  0.50  67 Kg N per ha  67 Kg N per ha   
428     USA     19.6    976.0  0.74  0.50  67 Kg N per ha  67 Kg N per ha   

                Tillage_C             Tillage_T CoverCropGroup GrainCropGroup  \
424  Conventional tillage  Conservation tillage             LL           Corn   
425  Conventional tillage  Conservation tillage         Legume           Corn   
426  Conventional tillage  Conservation tillage         Legume           Corn   
427  Conventional tillage  Conservation tillage             LL           Corn   
428  Conventional tillage  Conservation tillage         Legume           Corn   

    Conservation_Type  
424                CC  
42

In [ ]:
# prompt: create count of conservation_Type

conservation_type_counts = df_new_cleaned['Conservation_Type'].value_counts()
print("Counts of each Conservation_Type:")
conservation_type_counts

Counts of each Conservation_Type:


,count
Conservation_Type,
CC,277
CC-NT,124
OF-CC,13


So much temperature and precipitation data is missing, test using country instead compared to using much lower amount of records with Tannual and Pannual data. Later version can get an estimate of these values from the lat/long to use more of the data.

In [ ]:
# prompt: # prompt: duplicate records to have all fields with '_T' in seperate records to '_C'. CoverCropGroup and Conservation_Type should be 'None' for all '_C' records

# Duplicate records for '_T' and '_C'
df_t = df_new_cleaned.copy()
df_c = df_new_cleaned.copy()

# Rename columns for '_T' records
df_t.rename(columns={'OC_T': 'OC',
                     'Tillage_T': 'Tillage'}, inplace=True)
df_t = df_t.drop(columns=['OC_C', 'Tillage_C'])
df_t['Type'] = '_T'


# Rename columns for '_C' records and set 'CoverCropGroup' and 'Conservation_Type' to None
df_c.rename(columns={'OC_C': 'OC',
                     'Tillage_C': 'Tillage'}, inplace=True)
df_c = df_c.drop(columns=['OC_T', 'Tillage_T'])
df_c['Type'] = '_C'
df_c['CoverCropGroup'] = 'None'
df_c['Conservation_Type'] = 'None'


# Concatenate the two dataframes
df_combined = pd.concat([df_t, df_c], ignore_index=True)

# Display the first few rows of the combined DataFrame
print(df_combined.head())
print(df_combined.tail())

# Check the number of rows in the combined DataFrame
print(f"Number of rows in combined DataFrame: {len(df_combined)}")
print(f"Number of rows in df_new_cleaned: {len(df_new_cleaned)}")

  Country  Tannual  Pannual    OC   Fertilization               Tillage  \
0     USA     19.6    976.0  0.74   0 Kg N per ha  Conservation tillage   
1     USA     19.6    976.0  0.70   0 Kg N per ha  Conservation tillage   
2     USA     19.6    976.0  0.59   0 Kg N per ha  Conservation tillage   
3     USA     19.6    976.0  0.73  67 Kg N per ha  Conservation tillage   
4     USA     19.6    976.0  0.74  67 Kg N per ha  Conservation tillage   

  CoverCropGroup GrainCropGroup Conservation_Type Type  
0             LL           Corn                CC   _T  
1         Legume           Corn                CC   _T  
2         Legume           Corn                CC   _T  
3             LL           Corn                CC   _T  
4         Legume           Corn                CC   _T  
    Country  Tannual  Pannual    OC Fertilization              Tillage  \
823     USA     17.3   1958.0  0.87  C-0-N-0-OC-0  Traditional tillage   
824     USA     17.3   1958.0  0.86  C-0-N-0-OC-0  Traditio

In [ ]:
# prompt: list unique values of GrainCropGroup

print(df_combined['GrainCropGroup'].unique())

['Corn' 'Arable' 'Soybean' 'CS' 'Vegetable' 'CV' 'CO' 'MTT' 'CW' 'Wheat'
 'WO' 'AVG']


In [ ]:
# prompt: show unique tillage types

print(df_combined['Tillage'].unique())

['Conservation tillage' 'no tillage' nan 'spring tillage'
 'conventional tillage' 'Strip tillage' 'Chisel tillage'
 'no tillage/chisel plowing/moldboard plowing mixed' 'chisel plowing'
 'moldboard plowing mixed' 'No tillage' 'CT' 'no data' 'No till'
 'No-tillage' 'Conservational tillage' 'Conventional tillage'
 'CT / NT (not clear which one, influence found to be not significant)'
 'Disc till' 'Harrowing' 'mouldboard ploughing' 'Traditional tillage']


In [ ]:
print(f"Number of NA's in Tillage column: {df_combined['Tillage'].isnull().sum()}")

Number of NA's in Tillage column: 101


In [ ]:
# prompt: check how many 'no data' records there are for Tillage

print(f"Number of 'no data' records for Tillage: {(df_combined['Tillage'] == 'no data').sum()}")

Number of 'no data' records for Tillage: 13


For Tillage, null values, and 'no data' can be changed to Unknown. Fertilization needs to be cleaned, but the data is confusing and incomplete, so I will try the analysis without this field.

In [ ]:
# prompt: make all null and 'no data' records for Tillage as 'Unknown'

df_combined['Tillage'] = df_combined['Tillage'].fillna('Unknown')
df_combined['Tillage'] = df_combined['Tillage'].replace('no data', 'Unknown')

print(df_combined['Tillage'].unique())
print(f"Number of 'Unknown' records for Tillage after cleaning: {(df_combined['Tillage'] == 'Unknown').sum()}")

['Conservation tillage' 'no tillage' 'Unknown' 'spring tillage'
 'conventional tillage' 'Strip tillage' 'Chisel tillage'
 'no tillage/chisel plowing/moldboard plowing mixed' 'chisel plowing'
 'moldboard plowing mixed' 'No tillage' 'CT' 'No till' 'No-tillage'
 'Conservational tillage' 'Conventional tillage'
 'CT / NT (not clear which one, influence found to be not significant)'
 'Disc till' 'Harrowing' 'mouldboard ploughing' 'Traditional tillage']
Number of 'Unknown' records for Tillage after cleaning: 114


In [ ]:
# prompt: Make 'no tillage', 'No tillage', 'No till', 'No-tillage', 'CT / NT', as ''None'

tillage_replacements = {
    'no tillage': 'None',
    'No tillage': 'None',
    'No till': 'None',
    'No-tillage': 'None',
    'CT / NT (not clear which one, influence found to be not significant)': 'None'
}

df_combined['Tillage'] = df_combined['Tillage'].replace(tillage_replacements)

print(df_combined['Tillage'].unique())


['Conservation tillage' 'None' 'Unknown' 'spring tillage'
 'conventional tillage' 'Strip tillage' 'Chisel tillage'
 'no tillage/chisel plowing/moldboard plowing mixed' 'chisel plowing'
 'moldboard plowing mixed' 'CT' 'Conservational tillage'
 'Conventional tillage' 'Disc till' 'Harrowing' 'mouldboard ploughing'
 'Traditional tillage']


In [ ]:
conventional_replacements = {
    'spring tillage': 'Conventional',
    'conventional tillage': 'Conventional',
    'moldboard plowing mixed': 'Conventional',
    'CT': 'Conventional',
    'Conventional tillage':'Conventional',
    'Disc till':'Conventional',
    'Harrowing':'Conventional',
    'mouldboard ploughing':'Conventional',
    'Traditional tillage':'Conventional'
}

reduced_replacements = {
    'Conservation tillage': 'Reduced',
    'Strip tillage': 'Reduced',
    'Chisel tillage': 'Reduced',
    'no tillage/chisel plowing/moldboard plowing mixed': 'Reduced',
    'chisel plowing':'Reduced',
    'Conservational tillage':'Reduced'
}

df_combined['Tillage'] = df_combined['Tillage'].replace(conventional_replacements)
df_combined['Tillage'] = df_combined['Tillage'].replace(reduced_replacements)

print(df_combined['Tillage'].unique())

['Reduced' 'None' 'Unknown' 'Conventional']


In [ ]:
df_combined.head(5)

,Country,Tannual,Pannual,OC,Fertilization,Tillage,CoverCropGroup,GrainCropGroup,Conservation_Type,Type
0,USA,19.6,976.0,0.74,0 Kg N per ha,Reduced,LL,Corn,CC,_T
1,USA,19.6,976.0,0.70,0 Kg N per ha,Reduced,Legume,Corn,CC,_T
2,USA,19.6,976.0,0.59,0 Kg N per ha,Reduced,Legume,Corn,CC,_T
3,USA,19.6,976.0,0.73,67 Kg N per ha,Reduced,LL,Corn,CC,_T
4,USA,19.6,976.0,0.74,67 Kg N per ha,Reduced,Legume,Corn,CC,_T


In [ ]:
print(df_combined['CoverCropGroup'].unique())

['LL' 'Legume' 'Rye' 'Grass' 'Brassica' 'LG' 'Mixed' 'BG' 'Not_available'
 'AVG' 'None']


LL is Legume
LG, BG, and AVG are Mixed
Not_available is Unknown (consistent with Tillage)

In [ ]:
df_combined['CoverCropGroup'] = df_combined['CoverCropGroup'].replace('LL', 'Legume')
df_combined['CoverCropGroup'] = df_combined['CoverCropGroup'].replace(['LG', 'BG', 'AVG'], 'Mixed')
df_combined['CoverCropGroup'] = df_combined['CoverCropGroup'].replace('Not_available', 'Unknown')

print(df_combined['CoverCropGroup'].unique())


['Legume' 'Rye' 'Grass' 'Brassica' 'Mixed' 'Unknown' 'None']


In [ ]:
print(df_combined['GrainCropGroup'].unique())

['Corn' 'Arable' 'Soybean' 'CS' 'Vegetable' 'CV' 'CO' 'MTT' 'CW' 'Wheat'
 'WO' 'AVG']


Monoculture: Corn, Soybean, Wheat, Arable
Mono-Rotation: CS, CO, MTT, CW, WO
Unknown: AVG


In [ ]:
df_combined['GrainCropGroup'] = df_combined['GrainCropGroup'].replace(['Corn', 'Soybean', 'Wheat','Arable'], 'Monoculture')
df_combined['GrainCropGroup'] = df_combined['GrainCropGroup'].replace(['CS', 'CO', 'MTT', 'CW', 'WO'], 'Monoculture-rotation')
df_combined['GrainCropGroup'] = df_combined['GrainCropGroup'].replace('AVG', 'Unknown')
df_combined['GrainCropGroup'] = df_combined['GrainCropGroup'].replace('CV', 'Vegetable')


print(df_combined['GrainCropGroup'].unique())

['Monoculture' 'Monoculture-rotation' 'Vegetable' 'Unknown']


In [ ]:
from google.colab import files

# Export the DataFrame to a CSV file
df_combined.to_csv('df_combined.csv', index=False)

# Download the CSV file
files.download('df_combined.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>